In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import subprocess
import os
import sys
from pathlib import Path

def get_video_duration(video_path):
    """Get the duration of a video in seconds using ffprobe."""
    try:
        cmd = [
            'ffprobe', '-v', 'quiet', '-print_format', 'json',
            '-show_format', video_path
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)

        import json
        data = json.loads(result.stdout)
        duration = float(data['format']['duration'])
        return duration
    except (subprocess.CalledProcessError, KeyError, ValueError) as e:
        print(f"Error getting video duration: {e}")
        return None

def segment_video(input_path, output_dir=None, segment_duration=30):
    """
    Segment a video into chunks of specified duration.

    Args:
        input_path (str): Path to the input video file
        output_dir (str): Directory to save segments (optional)
        segment_duration (int): Duration of each segment in seconds (default: 30)
    """
    input_path = Path(input_path)

    if not input_path.exists():
        print(f"Error: Input file '{input_path}' does not exist.")
        return False

    # Set output directory
    if output_dir is None:
        output_dir = input_path.parent / f"{input_path.stem}_segments"
    else:
        output_dir = Path(output_dir)

    # Create output directory if it doesn't exist
    output_dir.mkdir(exist_ok=True)

    # Get video duration
    total_duration = get_video_duration(str(input_path))
    if total_duration is None:
        return False

    # Calculate number of segments
    num_segments = int(total_duration // segment_duration) + (1 if total_duration % segment_duration > 0 else 0)

    print(f"Video duration: {total_duration:.2f} seconds")
    print(f"Segmenting into {num_segments} chunks of {segment_duration} seconds each")
    print(f"Output directory: {output_dir}")

    # Get file extension
    file_extension = input_path.suffix

    success_count = 0

    for i in range(num_segments):
        start_time = i * segment_duration
        output_filename = f"{input_path.stem}_segment_{i+1:03d}{file_extension}"
        output_path = output_dir / output_filename

        # FFmpeg command for segmentation
        cmd = [
            'ffmpeg',
            '-i', str(input_path),
            '-ss', str(start_time),
            '-t', str(segment_duration),
            '-c', 'copy',  # Copy streams without re-encoding for speed
            '-avoid_negative_ts', 'make_zero',
            str(output_path),
            '-y'  # Overwrite output files without asking
        ]

        try:
            print(f"Creating segment {i+1}/{num_segments}: {output_filename}")
            subprocess.run(cmd, check=True, capture_output=True)
            success_count += 1
        except subprocess.CalledProcessError as e:
            print(f"Error creating segment {i+1}: {e}")
            continue

    print(f"\nSegmentation complete! {success_count}/{num_segments} segments created successfully.")
    return success_count, num_segments

def segment_video_with_reencoding(input_path, output_dir=None, segment_duration=30, video_codec='libx264', audio_codec='aac'):
    """
    Segment a video with re-encoding (slower but more compatible).

    Args:
        input_path (str): Path to the input video file
        output_dir (str): Directory to save segments (optional)
        segment_duration (int): Duration of each segment in seconds (default: 30)
        video_codec (str): Video codec to use (default: libx264)
        audio_codec (str): Audio codec to use (default: aac)
    """
    input_path = Path(input_path)

    if not input_path.exists():
        print(f"Error: Input file '{input_path}' does not exist.")
        return False

    # Set output directory
    if output_dir is None:
        output_dir = input_path.parent / f"{input_path.stem}_segments_reencoded"
    else:
        output_dir = Path(output_dir)

    # Create output directory if it doesn't exist
    output_dir.mkdir(exist_ok=True)

    # Get video duration
    total_duration = get_video_duration(str(input_path))
    if total_duration is None:
        return False

    # Calculate number of segments
    num_segments = int(total_duration // segment_duration) + (1 if total_duration % segment_duration > 0 else 0)

    print(f"Video duration: {total_duration:.2f} seconds")
    print(f"Segmenting with re-encoding into {num_segments} chunks of {segment_duration} seconds each")
    print(f"Output directory: {output_dir}")

    success_count = 0

    for i in range(num_segments):
        start_time = i * segment_duration
        output_filename = f"{input_path.stem}_segment_{i+1:03d}.mp4"
        output_path = output_dir / output_filename

        # FFmpeg command with re-encoding
        cmd = [
            'ffmpeg',
            '-i', str(input_path),
            '-ss', str(start_time),
            '-t', str(segment_duration),
            '-c:v', video_codec,
            '-c:a', audio_codec,
            '-preset', 'fast',  # Faster encoding
            str(output_path),
            '-y'
        ]

        try:
            print(f"Creating segment {i+1}/{num_segments}: {output_filename}")
            subprocess.run(cmd, check=True, capture_output=True)
            success_count += 1
        except subprocess.CalledProcessError as e:
            print(f"Error creating segment {i+1}: {e}")
            continue

    print(f"\nSegmentation complete! {success_count}/{num_segments} segments created successfully.")
    return success_count, num_segments

In [10]:
import os
datapath = "/content/drive/MyDrive/CSE253_Assignment2/"
filenames = os.listdir(datapath + 'music_videos/')
filenames[0]
segmented_videos = []

In [22]:
def seg_video(input_path, output_dir=None, segment_duration=30):
  success, num_segs = segment_video(input_video, output_dir, segment_duration)

  if not success:
      print("\nFast segmentation failed. Trying with re-encoding...")
      success, num_segs = segment_video_with_reencoding(input_video, output_dir, segment_duration)

  return success, num_segs

filename_splits = {}
total_durations = 0
for file in filenames:
  print("Segmenting ", file)
  input_video = datapath + 'music_videos/' + file
  output_dir = '/content/segmented_videos/'
  segment_duration = 30
  success, num_segs = seg_video(input_video, output_dir, segment_duration)
  total_durations += get_video_duration(str(input_video))
  print(f'Successes: {success} Num segments: {num_segs}')
  if success != num_segs:
    print("Fast segmentation failed. Trying with re-encoding...")
    success, num_segs = segment_video_with_reencoding(input_video, output_dir, segment_duration)
  else:
    segmented_videos.append(file)
    filename_splits[file] = num_segs
  print(f'{filename_splits}. Total Duration: {total_durations}')

with open(datapath + 'segmented_videos.txt', 'w') as f:
    f.write("\n".join(segmented_videos))

Segmenting  BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA].mp4
Video duration: 216.85 seconds
Segmenting into 8 chunks of 30 seconds each
Output directory: /content/segmented_videos
Creating segment 1/8: BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA]_segment_001.mp4
Creating segment 2/8: BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA]_segment_002.mp4
Creating segment 3/8: BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA]_segment_003.mp4
Creating segment 4/8: BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA]_segment_004.mp4
Creating segment 5/8: BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA]_segment_005.mp4
Creating segment 6/8: BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA]_segment_006.mp4
Creating segment 7/8: BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA]_segment_007.mp4
Creating segment 8/8: BLACKPINK - '마지막처럼 (AS IF 

In [ ]:
filenames = os.listdir(datapath + '/segmented_videos/')
len(filenames)

959

In [23]:
total_durations

27183.855397

In [31]:
video_to_artist = {
    "BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA].mp4": "BLACKPINK",
    "I CAN SWING MY SWORD! - Minecraft Song [eN7dYDYfvVg].mp4": "Unknown Artist",
    "Elijah - ＂Woe to him!＂ [VyDHbGEi6Ms].mp4": "Elijah",
    "20th Century Fox Shitty Flute Sheet Music [oZG7W_nWVV4].mp4": "Unknown Artist",
    "queeencard.mp4": "Unknown Artist",
    "BTS (방탄소년단) LOVE YOURSELF 承 Her 'Serendipity' Comeback Trailer [BEMaH9Sm3lQ].mp4": "BTS",
    "[MV] BTS(방탄소년단) _ I NEED U [NMdTd9e-LEI].mp4": "BTS",
    "BTS (방탄소년단) '호르몬전쟁' Official MV [Hhph0_CdUHg].mp4": "BTS",
    "[MV] BTS(방탄소년단) _ DOPE(쩔어) [BVwAVbKYYeM].mp4": "BTS",
    "BTS (방탄소년단) '상남자 (Boy In Luv)' Official MV [LHHjp3IMjMk].mp4": "BTS",
    "BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity' Comeback Trailer [p8npDG2ulKQ].mp4": "BTS",
    "black_magic.mp4": "Unknown Artist",
    "BTS (방탄소년단) LOVE YOURSELF 結 Answer 'Epiphany' Comeback Trailer [fIkZOLsnoqY].mp4": "BTS",
    "BLACKPINK - 'Lovesick Girls' M⧸V [dyRsYk0LyA8].mp4": "BLACKPINK",
    "oh_my_god.mp4": "Unknown Artist",
    "[MV] BTS(방탄소년단) _ War of Hormone(호르몬 전쟁) [XQmpVHUi-0A].mp4": "BTS",
    "BTS (방탄소년단) 'No More Dream' Official MV [rBG5L7UsUxA].mp4": "BTS",
    "Idina Menzel - Let It Go (from Frozen) (Official Video) [YVVTZgwYwVo].mp4": "Idina Menzel",
    "BTS (방탄소년단) '봄날 (Spring Day)' Official MV [xEeFrLSkMm8].mp4": "BTS",
    "BLACKPINK - '뚜두뚜두 (DDU-DU DDU-DU)' M⧸V [IHNzOHi8sJs].mp4": "BLACKPINK",
    "BTS (방탄소년단) 'No More Dream' Official MV (Choreography Version) [l15_-4-_QO8].mp4": "BTS",
    "BTS (방탄소년단) MAP OF THE SOUL ： PERSONA 'Persona' Comeback Trailer [M9Uy0opVF3s].mp4": "BTS",
    "Queen – Bohemian Rhapsody (Official Video Remastered) [fJ9rUzIMcZQ].mp4": "Queen",
    "superbass.mp4": "Unknown Artist",
    "BLACKPINK - 'How You Like That' M⧸V [ioNng23DkIM].mp4": "BLACKPINK",
    "BLACKPINK - 'Ice Cream (with Selena Gomez)' M⧸V [vRXZj0DzXIA].mp4": "BLACKPINK",
    "BTS (방탄소년단) '작은 것들을 위한 시 (Boy With Luv) (feat. Halsey)' Official MV [XsX3ATc3FbA].mp4": "BTS",
    "Rossini, William Tell (Overture), Animated Graphical Score [1k8sGztMsKA].mp4": "Rossini",
    "bidi_bidi_bom_bom.mp4": "Unknown Artist",
    "me_rehuso.mp4": "Unknown Artist",
    "BTS (방탄소년단) '상남자 (Boy In Luv)' Official MV (Choreography Version) [4XyPdnTz_Q0].mp4": "BTS",
    "BTS (방탄소년단) 'Danger' Official MV [43r6lXilbcQ].mp4": "BTS",
    "BLACKPINK - '불장난 (PLAYING WITH FIRE)' M⧸V [9pdj4iJD08s].mp4": "BLACKPINK",
    "bad_romance.mp4": "Unknown Artist",
    "BTS (방탄소년단) 'Not Today' Official MV [9DwzBICPhdM].mp4": "BTS",
    "shape_of_you.mp4": "Unknown Artist",
    "girlfriend.mp4": "Unknown Artist",
    "BTS (방탄소년단) 'FAKE LOVE' Official MV [7C2z4GqqS5E].mp4": "BTS",
    "never_gonna_give_you_up.mp4": "Unknown Artist",
    "[MV] BTS(방탄소년단) _ Just One Day(하루만) [DTcKkcyS410].mp4": "BTS",
    "BTS 'FOR YOU' Official MV [TTG6nxwdhyA].mp4": "BTS",
    "primadonna.mp4": "Unknown Artist",
    "no_tears_left_to_cry.mp4": "Unknown Artist",
    "we_dont_talk_anymore.mp4": "Unknown Artist",
    "Charlie Puth - We Don't Talk Anymore (feat. Selena Gomez) [Official Video] [3AtDnEC4zak].mp4": "Charlie Puth",
    "Mendelssohn, Violin Concerto complete 1st mvt, E minor, Allegro molto appassionato [PxdPj6vm1Pg].mp4": "Mendelssohn",
    "feel_my_rhythm.mp4": "Unknown Artist",
    "BLACKPINK - 'Shut Down' M⧸V [POe9SOEKotk].mp4": "BLACKPINK",
    "BTS (방탄소년단) 'IDOL' Official MV [pBuZEGYXA6E].mp4": "BTS",
    "cosmic.mp4": "Unknown Artist",
    "BLACKPINK - 'STAY' M⧸V [FzVR_fymZw4].mp4": "BLACKPINK",
    "calm_down.mp4": "Unknown Artist",
    "BLACKPINK - 'Kill This Love' M⧸V [2S24-y0Ij3Y].mp4": "BLACKPINK",
    "BLACKPINK - 'Pink Venom' M⧸V [gQlMMD8auMs].mp4": "BLACKPINK",
    "Charlie Puth - Light Switch [Official Music Video] [WFsAon_TWPQ].mp4": "Charlie Puth",
    "BTS (防弾少年団) 'Airplane pt.2 -Japanese ver.-' Official MV [CxnJf0tWu48].mp4": "BTS",
    "russian_roulette.mp4": "Unknown Artist",
    "all_of_me.mp4": "Unknown Artist",
    "10 Minute Timer [MINECRAFT] 🟩 [t8afYsVz_6o].mp4": "Unknown Artist",
    "9_to_5.mp4": "Unknown Artist",
    "BTS (방탄소년단) 'Save ME' Official MV [GZjt_sA2eso].mp4": "BTS",
    "Sabrina Carpenter - Espresso (Official Video) [eVli-tstM5E].mp4": "Sabrina Carpenter",
    "apt.mp4": "Unknown Artist",
    "[MV] BTS(방탄소년단) _ FIRE (불타오르네) [ALj5MKjy2BU].mp4": "BTS",
    "maria.mp4": "Unknown Artist",
    "red_wine_supernova.mp4": "Unknown Artist",
    "Bach, Double Violin Concerto in D Minor, 1st mvt. BWV 1043 [x0daRRpLRJ4].mp4": "Bach",
    "20th Century Fox bad flute [2IRcM9qwDwo].mp4": "Unknown Artist",
    "Debussy, Clair de lune (piano music) [LlvUepMa31o].mp4": "Debussy",
    "BLACKPINK - '휘파람 (WHISTLE)' M⧸V [dISNgvVpWlo].mp4": "BLACKPINK",
    "BLACKPINK - '붐바야 (BOOMBAYAH)' M⧸V [bwmSjveL3Lc].mp4": "BLACKPINK",
    "Mozart, Eine Kleine Nachtmusik [tSL5-wxgvFY].mp4": "Mozart",
    "Emerson String Quartet： Shostakovich Quartet No. 8 in C minor, Op. 110 [41HIXtBElH4].mp4": "Shostakovich",
    "BTS (방탄소년단) 'DNA' Official MV [MBdVXkSdhwU].mp4": "BTS",
    "psycho.mp4": "Unknown Artist",
    "BTS (방탄소년단) MAP OF THE SOUL ： 7 'Interlude ： Shadow' Comeback Trailer [PV1gCvzpSy0].mp4": "BTS",
    "Mozart, Piano Concerto No. 20 in D minor (K. 466), 1st movement [7QgOBbKl0eY].mp4": "Mozart",
    "BTS (방탄소년단) '피 땀 눈물 (Blood Sweat & Tears)' Official MV [hmE9f-TEutc].mp4": "BTS",
    "BTS (방탄소년단) 'We Are Bulletproof Pt.2' Official MV [lE9lkSdtZeQ].mp4": "BTS",
    "Beethoven, Piano Sonata No. 14 in C-sharp minor (opus 27 no. 2, 'Moonlight') 1. Adagio sostenuto [oFSRs7iqAv8].mp4": "Beethoven",
    "Beethoven, Piano Sonata No. 8 in C minor (opus 13, ＂Pathetique＂) 3. RONDO. Allegro. [pSABCXv1kQw].mp4": "Beethoven",
    "Barbie as The Princess and The Pauper - How Can I Refuse [5otYUHbaJX8].mp4": "Unknown Artist",
    "Schubert, Piano Trio in E-flat major, 2nd mvt. [tKZBmMu06Fg].mp4": "Schubert",
    "Chopin, Fantasie-Impromptu, opus 66 [APQ2RKECMW8].mp4": "Chopin",
    "BTS (방탄소년단) 'N.O' Official MV [r5GaAEHvHj0].mp4": "BTS",
    "Mozart, Queen of the Night (aria from Magic Flute, K. 620) © [qLSzKrr-O-4].mp4": "Mozart",
    "BTS (방탄소년단) '하루만 (Just one day)' Official MV [dzAwJW107nk].mp4": "BTS",
    "BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Official MV [kTlv5_Bs8aw].mp4": "BTS",
    "Titanic Bad Flute Cover [BG6EtT-mReM].mp4": "Unknown Artist",
    "Bach, ＂Little＂ Fugue in G minor, Organ [ddbxFi3-UO4].mp4": "Bach",
    "Beethoven, 5th Symphony, 1st mvt. (＂fisheye＂) © [Qglck7rpI3w].mp4": "Beethoven",
    "me_enamore.mp4": "Unknown Artist",
    "gnarly.mp4": "Unknown Artist",
    "Charli xcx - party 4 u (official video) [agu22bqGHto].mp4": "Charli XCX",
    "Chopin, Nocturne in B Major, opus 32 no.1, Piano Solo (animated score) [tIZPrHl-kek].mp4": "Chopin",
    "Numb (Official Music Video) [4K UPGRADE] – Linkin Park.mp4": "Linkin Park",
    "Green Day - Boulevard Of Broken Dreams [Official Music Video] [4K Upgrade].mp4": "Green Day",
    "Green Day - American Idiot [Official Music Video] [4K Upgrade].mp4": "Green Day",
    "Queen - Don't Stop Me Now (Official Video).mp4": "Queen",
    "Journey - Don't Stop Believin' (Official Video RCF).mp4": "Journey",
    "Journey - Separate Ways (Worlds Apart) (Official HD Video - 1983).mp4": "Journey",
    "Dragonforce - Through the Fire and Flames [OFFICIAL VIDEO].mp4": "Dragonforce",
    "Billy Joel - We Didn't Start the Fire (Official HD Video).mp4": "Billy Joel"
}



In [34]:
for key in video_to_artist.keys():
  if key not in filename_splits.keys():
    print(key)

BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA].mp4
BTS (방탄소년단) LOVE YOURSELF 承 Her 'Serendipity' Comeback Trailer [BEMaH9Sm3lQ].mp4
[MV] BTS(방탄소년단) _ I NEED U [NMdTd9e-LEI].mp4
BTS (방탄소년단) '호르몬전쟁' Official MV [Hhph0_CdUHg].mp4
[MV] BTS(방탄소년단) _ DOPE(쩔어) [BVwAVbKYYeM].mp4
BTS (방탄소년단) '상남자 (Boy In Luv)' Official MV [LHHjp3IMjMk].mp4
BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity' Comeback Trailer [p8npDG2ulKQ].mp4
BTS (방탄소년단) LOVE YOURSELF 結 Answer 'Epiphany' Comeback Trailer [fIkZOLsnoqY].mp4
[MV] BTS(방탄소년단) _ War of Hormone(호르몬 전쟁) [XQmpVHUi-0A].mp4
BTS (방탄소년단) 'No More Dream' Official MV [rBG5L7UsUxA].mp4
BTS (방탄소년단) '봄날 (Spring Day)' Official MV [xEeFrLSkMm8].mp4
BLACKPINK - '뚜두뚜두 (DDU-DU DDU-DU)' M⧸V [IHNzOHi8sJs].mp4
BTS (방탄소년단) 'No More Dream' Official MV (Choreography Version) [l15_-4-_QO8].mp4
BTS (방탄소년단) MAP OF THE SOUL ： PERSONA 'Persona' Comeback Trailer [M9Uy0opVF3s].mp4
BTS (방탄소년단) '작은 것들을 위한 시 (Boy With Luv) (feat. Halsey)' Official MV [XsX3ATc3FbA].mp4
BTS (방

In [28]:
import numpy as np
np.unique(list(video_to_artist.values()), return_counts = True)

(array(['BLACKPINK', 'BTS', 'Bach', 'Beethoven', 'Billy Joel',
        'Charli XCX', 'Charlie Puth', 'Chopin', 'Debussy', 'Dragonforce',
        'Elijah', 'Green Day', 'Idina Menzel', 'Journey', 'Linkin Park',
        'Mendelssohn', 'Mozart', 'Queen', 'Rossini', 'Sabrina Carpenter',
        'Schubert', 'Shostakovich', 'Unknown Artist'], dtype='<U17'),
 array([12, 30,  2,  3,  1,  1,  2,  2,  1,  1,  1,  2,  1,  2,  1,  1,  3,
         2,  1,  1,  1,  1, 31]))

In [29]:
artist, counts = np.unique(list(video_to_artist.values()), return_counts = True)

for i, art in enumerate(artist):
  print(f'{art} : {counts[i]}')

BLACKPINK : 12
BTS : 30
Bach : 2
Beethoven : 3
Billy Joel : 1
Charli XCX : 1
Charlie Puth : 2
Chopin : 2
Debussy : 1
Dragonforce : 1
Elijah : 1
Green Day : 2
Idina Menzel : 1
Journey : 2
Linkin Park : 1
Mendelssohn : 1
Mozart : 3
Queen : 2
Rossini : 1
Sabrina Carpenter : 1
Schubert : 1
Shostakovich : 1
Unknown Artist : 31


In [36]:
video_to_segments = {
    "BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA].mp4": 8,
    "I CAN SWING MY SWORD! - Minecraft Song [eN7dYDYfvVg].mp4": 4,
    "Elijah - \"Woe to him!\" [VyDHbGEi6Ms].mp4": 4,
    "20th Century Fox Shitty Flute Sheet Music [oZG7W_nWVV4].mp4": 1,
    "queeencard.mp4": 9,
    "BTS (방탄소년단) LOVE YOURSELF 承 Her 'Serendipity' Comeback Trailer [BEMaH9Sm3lQ].mp4": 6,
    "[MV] BTS(방탄소년단) _ I NEED U [NMdTd9e-LEI].mp4": 8,
    "BTS (방탄소년단) '호르몬전쟁' Official MV [Hhph0_CdUHg].mp4": 10,
    "[MV] BTS(방탄소년단) _ DOPE(쩔어) [BVwAVbKYYeM].mp4": 9,
    "BTS (방탄소년단) '상남자 (Boy In Luv)' Official MV [LHHjp3IMjMk].mp4": 10,
    "BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity' Comeback Trailer [p8npDG2ulKQ].mp4": 8,
    "black_magic.mp4": 8,
    "BTS (방탄소년단) LOVE YOURSELF 結 Answer 'Epiphany' Comeback Trailer [fIkZOLsnoqY].mp4": 9,
    "BLACKPINK - 'Lovesick Girls' M⧸V [video_to_artist = {
    "BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA].mp4": "BLACKPINK",
    "I CAN SWING MY SWORD! - Minecraft Song [eN7dYDYfvVg].mp4": "Unknown Artist",
    "Elijah - \"Woe to him!\" [VyDHbGEi6Ms].mp4": "Elijah",
    "20th Century Fox Shitty Flute Sheet Music [oZG7W_nWVV4].mp4": "Unknown Artist",
    "queeencard.mp4": "Unknown Artist",
    "BTS (방탄소년단) LOVE YOURSELF 承 Her 'Serendipity' Comeback Trailer [BEMaH9Sm3lQ].mp4": "BTS",
    "[MV] BTS(방탄소년단) _ I NEED U [NMdTd9e-LEI].mp4": "BTS",
    "BTS (방탄소년단) '호르몬전쟁' Official MV [Hhph0_CdUHg].mp4": "BTS",
    "[MV] BTS(방탄소년단) _ DOPE(쩔어) [BVwAVbKYYeM].mp4": "BTS",
    "BTS (방탄소년단) '상남자 (Boy In Luv)' Official MV [LHHjp3IMjMk].mp4": "BTS",
    "BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity' Comeback Trailer [p8npDG2ulKQ].mp4": "BTS",
    "black_magic.mp4": "Unknown Artist",
    "BTS (방탄소년단) LOVE YOURSELF 結 Answer 'Epiphany' Comeback Trailer [fIkZOLsnoqY].mp4": "BTS",
    "BLACKPINK - 'Lovesick Girls' M⧸V [dyRsYk0LyA8].mp4": "BLACKPINK",
    "oh_my_god.mp4": "Unknown Artist",
    "[MV] BTS(방탄소년단) _ War of Hormone(호르몬 전쟁) [XQmpVHUi-0A].mp4": "BTS",
    "BTS (방탄소년단) 'No More Dream' Official MV [rBG5L7UsUxA].mp4": "BTS",
    "Idina Menzel - Let It Go (from Frozen) (Official Video) [YVVTZgwYwVo].mp4": "Idina Menzel",
    "BTS (방탄소년단) '봄날 (Spring Day)' Official MV [xEeFrLSkMm8].mp4": "BTS",
    "BLACKPINK - '뚜두뚜두 (DDU-DU DDU-DU)' M⧸V [IHNzOHi8sJs].mp4": "BLACKPINK",
    "BTS (방탄소년단) 'No More Dream' Official MV (Choreography Version) [l15_-4-_QO8].mp4": "BTS",
    "BTS (방탄소년단) MAP OF THE SOUL ： PERSONA 'Persona' Comeback Trailer [M9Uy0opVF3s].mp4": "BTS",
    "Queen – Bohemian Rhapsody (Official Video Remastered) [fJ9rUzIMcZQ].mp4": "Queen",
    "superbass.mp4": "Unknown Artist",
    "BLACKPINK - 'How You Like That' M⧸V [ioNng23DkIM].mp4": "BLACKPINK",
    "BLACKPINK - 'Ice Cream (with Selena Gomez)' M⧸V [vRXZj0DzXIA].mp4": "BLACKPINK",
    "BTS (방탄소년단) '작은 것들을 위한 시 (Boy With Luv) (feat. Halsey)' Official MV [XsX3ATc3FbA].mp4": "BTS",
    "Rossini, William Tell (Overture), Animated Graphical Score [1k8sGztMsKA].mp4": "Rossini",
    "bidi_bidi_bom_bom.mp4": "Unknown Artist",
    "me_rehuso.mp4": "Unknown Artist",
    "BTS (방탄소년단) '상남자 (Boy In Luv)' Official MV (Choreography Version) [4XyPdnTz_Q0].mp4": "BTS",
    "BTS (방탄소년단) 'Danger' Official MV [43r6lXilbcQ].mp4": "BTS",
    "BLACKPINK - '불장난 (PLAYING WITH FIRE)' M⧸V [9pdj4iJD08s].mp4": "BLACKPINK",
    "bad_romance.mp4": "Unknown Artist",
    "BTS (방탄소년단) 'Not Today' Official MV [9DwzBICPhdM].mp4": "BTS",
    "shape_of_you.mp4": "Unknown Artist",
    "girlfriend.mp4": "Unknown Artist",
    "BTS (방탄소년단) 'FAKE LOVE' Official MV [7C2z4GqqS5E].mp4": "BTS",
    "never_gonna_give_you_up.mp4": "Unknown Artist",
    "[MV] BTS(방탄소년단) _ Just One Day(하루만) [DTcKkcyS410].mp4": "BTS",
    "BTS 'FOR YOU' Official MV [TTG6nxwdhyA].mp4": "BTS",
    "primadonna.mp4": "Unknown Artist",
    "no_tears_left_to_cry.mp4": "Unknown Artist",
    "we_dont_talk_anymore.mp4": "Unknown Artist",
    "Charlie Puth - We Don't Talk Anymore (feat. Selena Gomez) [Official Video] [3AtDnEC4zak].mp4": "Charlie Puth",
    "Mendelssohn, Violin Concerto complete 1st mvt, E minor, Allegro molto appassionato [PxdPj6vm1Pg].mp4": "Mendelssohn",
    "feel_my_rhythm.mp4": "Unknown Artist",
    "BLACKPINK - 'Shut Down' M⧸V [POe9SOEKotk].mp4": "BLACKPINK",
    "BTS (방탄소년단) 'IDOL' Official MV [pBuZEGYXA6E].mp4": "BTS",
    "cosmic.mp4": "Unknown Artist",
    "BLACKPINK - 'STAY' M⧸V [FzVR_fymZw4].mp4": "BLACKPINK",
    "calm_down.mp4": "Unknown Artist",
    "BLACKPINK - 'Kill This Love' M⧸V [2S24-y0Ij3Y].mp4": "BLACKPINK",
    "BLACKPINK - 'Pink Venom' M⧸V [gQlMMD8auMs].mp4": "BLACKPINK",
    "Charlie Puth - Light Switch [Official Music Video] [WFsAon_TWPQ].mp4": "Charlie Puth",
    "BTS (防弾少年団) 'Airplane pt.2 -Japanese ver.-' Official MV [CxnJf0tWu48].mp4": "BTS",
    "russian_roulette.mp4": "Unknown Artist",
    "all_of_me.mp4": "Unknown Artist",
    "10 Minute Timer [MINECRAFT] 🟩 [t8afYsVz_6o].mp4": "Unknown Artist",
    "9_to_5.mp4": "Unknown Artist",
    "BTS (방탄소년단) 'Save ME' Official MV [GZjt_sA2eso].mp4": "BTS",
    "Sabrina Carpenter - Espresso (Official Video) [eVli-tstM5E].mp4": "Sabrina Carpenter",
    "apt.mp4": "Unknown Artist",
    "[MV] BTS(방탄소년단) _ FIRE (불타오르네) [ALj5MKjy2BU].mp4": "BTS",
    "maria.mp4": "Unknown Artist",
    "red_wine_supernova.mp4": "Unknown Artist",
    "Bach, Double Violin Concerto in D Minor, 1st mvt. BWV 1043 [x0daRRpLRJ4].mp4": "Bach",
    "20th Century Fox bad flute [2IRcM9qwDwo].mp4": "Unknown Artist",
    "Debussy, Clair de lune (piano music) [LlvUepMa31o].mp4": "Debussy",
    "BLACKPINK - '휘파람 (WHISTLE)' M⧸V [dISNgvVpWlo].mp4": "BLACKPINK",
    "BLACKPINK - '붐바야 (BOOMBAYAH)' M⧸V [bwmSjveL3Lc].mp4": "BLACKPINK",
    "Mozart, Eine Kleine Nachtmusik [tSL5-wxgvFY].mp4": "Mozart",
    "Emerson String Quartet： Shostakovich Quartet No. 8 in C minor, Op. 110 [41HIXtBElH4].mp4": "Shostakovich",
    "BTS (방탄소년단) 'DNA' Official MV [MBdVXkSdhwU].mp4": "BTS",
    "psycho.mp4": "Unknown Artist",
    "BTS (방탄소년단) MAP OF THE SOUL ： 7 'Interlude ： Shadow' Comeback Trailer [PV1gCvzpSy0].mp4": "BTS",
    "Mozart, Piano Concerto No. 20 in D minor (K. 466), 1st movement [7QgOBbKl0eY].mp4": "Mozart",
    "BTS (방탄소년단) '피 땀 눈물 (Blood Sweat & Tears)' Official MV [hmE9f-TEutc].mp4": "BTS",
    "BTS (방탄소년단) 'We Are Bulletproof Pt.2' Official MV [lE9lkSdtZeQ].mp4": "BTS",
    "Beethoven, Piano Sonata No. 14 in C-sharp minor (opus 27 no. 2, 'Moonlight') 1. Adagio sostenuto [oFSRs7iqAv8].mp4": "Beethoven",
    "Beethoven, Piano Sonata No. 8 in C minor (opus 13, \"Pathetique\") 3. RONDO. Allegro. [pSABCXv1kQw].mp4": "Beethoven",
    "Barbie as The Princess and The Pauper - How Can I Refuse [5otYUHbaJX8].mp4": "Unknown Artist",
    "Schubert, Piano Trio in E-flat major, 2nd mvt. [tKZBmMu06Fg].mp4": "Schubert",
    "Chopin, Fantasie-Impromptu, opus 66 [APQ2RKECMW8].mp4": "Chopin",
    "BTS (방탄소년단) 'N.O' Official MV [r5GaAEHvHj0].mp4": "BTS",
    "Mozart, Queen of the Night (aria from Magic Flute, K. 620) © [qLSzKrr-O-4].mp4": "Mozart",
    "BTS (방탄소년단) '하루만 (Just one day)' Official MV [dzAwJW107nk].mp4": "BTS",
    "BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Official MV [kTlv5_Bs8aw].mp4": "BTS",
    "Titanic Bad Flute Cover [BG6EtT-mReM].mp4": "Unknown Artist",
    "Bach, \"Little\" Fugue in G minor, Organ [ddbxFi3-UO4].mp4": "Bach",
    "Beethoven, 5th Symphony, 1st mvt. (\"fisheye\") © [Qglck7rpI3w].mp4": "Beethoven",
    "me_enamore.mp4": "Unknown Artist",
    "gnarly.mp4": "Unknown Artist",
    "Charli xcx - party 4 u (official video) [agu22bqGHto].mp4": "Charli XCX",
    "Chopin, Nocturne in B Major, opus 32 no.1, Piano Solo (animated score) [tIZPrHl-kek].mp4": "Chopin",
    "Numb (Official Music Video) [4K UPGRADE] – Linkin Park.mp4": "Linkin Park",
    "Green Day - Boulevard Of Broken Dreams [Official Music Video] [4K Upgrade].mp4": "Green Day",
    "Green Day - American Idiot [Official Music Video] [4K Upgrade].mp4": "Green Day",
    "Queen - Don't Stop Me Now (Official Video).mp4": "Queen",
    "Journey - Don't Stop Believin' (Official Video RCF).mp4": "Journey",
    "Journey - Separate Ways (Worlds Apart) (Official HD Video - 1983).mp4": "Journey",
    "Dragonforce - Through the Fire and Flames [OFFICIAL VIDEO].mp4": "Dragonforce",
    "Billy Joel - We Didn't Start the Fire (Official HD Video).mp4": "Billy Joel"
}

video_to_segments = {"BLACKPINK - '마지막처럼 (AS IF IT'S YOUR LAST)' M⧸V [Amq-qlqbjYA].mp4": 8, 'I CAN SWING MY SWORD! - Minecraft Song [eN7dYDYfvVg].mp4': 4, 'Elijah - ＂Woe to him!＂ [VyDHbGEi6Ms].mp4': 4, '20th Century Fox Shitty Flute Sheet Music [oZG7W_nWVV4].mp4': 1, 'queeencard.mp4': 9, "BTS (방탄소년단) LOVE YOURSELF 承 Her 'Serendipity' Comeback Trailer [BEMaH9Sm3lQ].mp4": 6, '[MV] BTS(방탄소년단) _ I NEED U [NMdTd9e-LEI].mp4': 8, "BTS (방탄소년단) '호르몬전쟁' Official MV [Hhph0_CdUHg].mp4": 10, '[MV] BTS(방탄소년단) _ DOPE(쩔어) [BVwAVbKYYeM].mp4': 9, "BTS (방탄소년단) '상남자 (Boy In Luv)' Official MV [LHHjp3IMjMk].mp4": 10, "BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity' Comeback Trailer [p8npDG2ulKQ].mp4": 8, 'black_magic.mp4': 8, "BTS (방탄소년단) LOVE YOURSELF 結 Answer 'Epiphany' Comeback Trailer [fIkZOLsnoqY].mp4": 9, "BLACKPINK - 'Lovesick Girls' M⧸V [dyRsYk0LyA8].mp4": 7, 'oh_my_god.mp4': 7, '[MV] BTS(방탄소년단) _ War of Hormone(호르몬 전쟁) [XQmpVHUi-0A].mp4': 10, "BTS (방탄소년단) 'No More Dream' Official MV [rBG5L7UsUxA].mp4": 10, 'Idina Menzel - Let It Go (from Frozen) (Official Video) [YVVTZgwYwVo].mp4': 8, "BTS (방탄소년단) '봄날 (Spring Day)' Official MV [xEeFrLSkMm8].mp4": 11, 'BLACKPINK - '뚜두뚜두 (DDU-DU DDU-DU)' M⧸V [IHNzOHi8sJs].mp4': 8, "BTS (방탄소년단) 'No More Dream' Official MV (Choreography Version) [l15_-4-_QO8].mp4": 9, "BTS (방탄소년단) MAP OF THE SOUL ： PERSONA 'Persona' Comeback Trailer [M9Uy0opVF3s].mp4": 6, 'Queen – Bohemian Rhapsody (Official Video Remastered) [fJ9rUzIMcZQ].mp4': 12, 'superbass.mp4': 8, "BLACKPINK - 'How You Like That' M⧸V [ioNng23DkIM].mp4": 7, "BLACKPINK - 'Ice Cream (with Selena Gomez)' M⧸V [vRXZj0DzXIA].mp4": 7, "BTS (방탄소년단) '작은 것들을 위한 시 (Boy With Luv) (feat. Halsey)' Official MV [XsX3ATc3FbA].mp4": 9, 'Rossini, William Tell (Overture), Animated Graphical Score [1k8sGztMsKA].mp4': 7, 'bidi_bidi_bom_bom.mp4': 7, 'me_rehuso.mp4': 8, "BTS (방탄소년단) '상남자 (Boy In Luv)' Official MV (Choreography Version) [4XyPdnTz_Q0].mp4": 8, "BTS (방탄소년단) 'Danger' Official MV [43r6lXilbcQ].mp4": 10, "BLACKPINK - '불장난 (PLAYING WITH FIRE)' M⧸V [9pdj4iJD08s].mp4": 7, 'bad_romance.mp4': 11, "BTS (방탄소년단) 'Not Today' Official MV [9DwzBICPhdM].mp4": 10, 'shape_of_you.mp4': 9, 'girlfriend.mp4': 8, "BTS (방탄소년단) 'FAKE LOVE' Official MV [7C2z4GqqS5E].mp4": 11, 'never_gonna_give_you_up.mp4': 8, '[MV] BTS(방탄소년단) _ Just One Day(하루만) [DTcKkcyS410].mp4': 9, "BTS 'FOR YOU' Official MV [TTG6nxwdhyA].mp4": 11, 'primadonna.mp4': 8, 'no_tears_left_to_cry.mp4': 8, 'we_dont_talk_anymore.mp4': 8, "Charlie Puth - We Don't Talk Anymore (feat. Selena Gomez) [Official Video] [3AtDnEC4zak].mp4": 8, 'Mendelssohn, Violin Concerto complete 1st mvt, E minor, Allegro molto appassionato [PxdPj6vm1Pg].mp4': 23, 'feel_my_rhythm.mp4': 8, 'BLACKPINK - 'Shut Down' M⧸V [POe9SOEKotk].mp4': 7, "BTS (방탄소년단) 'IDOL' Official MV [pBuZEGYXA6E].mp4": 8, 'cosmic.mp4': 8, "BLACKPINK - 'STAY' M⧸V [FzVR_fymZw4].mp4": 9, 'calm_down.mp4': 8, "BLACKPINK - 'Kill This Love' M⧸V [2S24-y0Ij3Y].mp4": 7, 'BLACKPINK - 'Pink Venom' M⧸V [gQlMMD8auMs].mp4': 7, 'Charlie Puth - Light Switch [Official Music Video] [WFsAon_TWPQ].mp4': 7, "BTS (防弾少年団) 'Airplane pt.2 -Japanese ver.-' Official MV [CxnJf0tWu48].mp4": 8, 'russian_roulette.mp4': 8, 'all_of_me.mp4': 11, '10 Minute Timer [MINECRAFT] 🟩 [t8afYsVz_6o].mp4': 21, '9_to_5.mp4': 6, "BTS (방탄소년단) 'Save ME' Official MV [GZjt_sA2eso].mp4": 8, 'Sabrina Carpenter - Espresso (Official Video) [eVli-tstM5E].mp4': 7, 'apt.mp4': 6, '[MV] BTS(방탄소년단) _ FIRE (불타오르네) [ALj5MKjy2BU].mp4': 10, 'maria.mp4': 8, 'red_wine_supernova.mp4': 8, 'Bach, Double Violin Concerto in D Minor, 1st mvt. BWV 1043 [x0daRRpLRJ4].mp4': 7, '20th Century Fox bad flute [2IRcM9qwDwo].mp4': 1, 'Debussy, Clair de lune (piano music) [LlvUepMa31o].mp4': 11, "BLACKPINK - '휘파람 (WHISTLE)' M⧸V [dISNgvVpWlo].mp4": 8, "BLACKPINK - '붐바야 (BOOMBAYAH)' M⧸V [bwmSjveL3Lc].mp4": 9, 'Mozart, Eine Kleine Nachtmusik [tSL5-wxgvFY].mp4': 12, 'Emerson String Quartet： Shostakovich Quartet No. 8 in C minor, Op. 110 [41HIXtBElH4].mp4': 44, "BTS (방탄소년단) 'DNA' Official MV [MBdVXkSdhwU].mp4": 9, 'psycho.mp4': 8, "BTS (방탄소년단) MAP OF THE SOUL ： 7 'Interlude ： Shadow' Comeback Trailer [PV1gCvzpSy0].mp4": 6, 'Mozart, Piano Concerto No. 20 in D minor (K. 466), 1st movement [7QgOBbKl0eY].mp4': 29, "BTS (방탄소년단) '피 땀 눈물 (Blood Sweat & Tears)' Official MV [hmE9f-TEutc].mp4": 13, "BTS (방탄소년단) 'We Are Bulletproof Pt.2' Official MV [lE9lkSdtZeQ].mp4": 8, "Beethoven, Piano Sonata No. 14 in C-sharp minor (opus 27 no. 2, 'Moonlight') 1. Adagio sostenuto [oFSRs7iqAv8].mp4": 16, 'Beethoven, Piano Sonata No. 8 in C minor (opus 13, ＂Pathetique＂) 3. RONDO. Allegro. [pSABCXv1kQw].mp4': 9, 'Barbie as The Princess and The Pauper - How Can I Refuse [5otYUHbaJX8].mp4': 6, 'Schubert, Piano Trio in E-flat major, 2nd mvt. [tKZBmMu06Fg].mp4': 19, 'Chopin, Fantasie-Impromptu, opus 66 [APQ2RKECMW8].mp4': 11, "BTS (방탄소년단) 'N.O' Official MV [r5GaAEHvHj0].mp4": 9, 'Mozart, Queen of the Night (aria from Magic Flute, K. 620) © [qLSzKrr-O-4].mp4': 6, "BTS (방탄소년단) '하루만 (Just one day)' Official MV [dzAwJW107nk].mp4": 9, "BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Official MV [kTlv5_Bs8aw].mp4": 10, 'Titanic Bad Flute Cover [BG6EtT-mReM].mp4': 9, 'Bach, ＂Little＂ Fugue in G minor, Organ [ddbxFi3-UO4].mp4': 8, 'Beethoven, 5th Symphony, 1st mvt. (＂fisheye＂) © [Qglck7rpI3w].mp4': 16, 'me_enamore.mp4': 8, 'gnarly.mp4': 5, 'Charli xcx - party 4 u (official video) [agu22bqGHto].mp4': 10, 'Chopin, Nocturne in B Major, opus 32 no.1, Piano Solo (animated score) [tIZPrHl-kek].mp4': 11, 'Numb (Official Music Video) [4K UPGRADE] – Linkin Park.mp4': 7, 'Green Day - Boulevard Of Broken Dreams [Official Music Video] [4K Upgrade].mp4': 10, 'Green Day - American Idiot [Official Music Video] [4K Upgrade].mp4': 7, "Queen - Don't Stop Me Now (Official Video).mp4": 8, "Journey - Don't Stop Believin' (Official Video RCF).mp4": 9, 'Journey - Separate Ways (Worlds Apart) (Official HD Video - 1983).mp4': 9, 'Dragonforce - Through the Fire and Flames [OFFICIAL VIDEO].mp4': 11, "Billy Joel - We Didn't Start the Fire (Official HD Video).mp4": 9}

# Create merged dictionary with artist names as keys and total segments as values
artist_to_total_segments = {}

for video_name, artist in video_to_artist.items():
    if video_name in video_to_segments:
        segments = video_to_segments[video_name]
        if artist in artist_to_total_segments:
            artist_to_total_segments[artist] += segments
        else:
            artist_to_total_segments[artist] = segments

# Sort by total segments (descending) for better readability
sorted_artists = dict(sorted(artist_to_total_segments.items(), key=lambda x: x[1], reverse=True))

print("Artist to Total Segments:")
print("=" * 40)
for artist, total_segments in sorted_artists.items():
    print(f"{artist}: {total_segments}")

print(f"\nFinal merged dictionary:")
print(artist_to_total_segments)

SyntaxError: unterminated string literal (detected at line 15) (<ipython-input-36-ebddf0e23e93>, line 15)